# Social Distancing and Mask Detectiong

In [1]:
# Import Libraries:

import cv2 
import numpy as np 
import face_detection 
from keras.models import load_model
from focal_loss import BinaryFocalLoss
from scipy.spatial import distance as dist
from keras.applications.resnet import preprocess_input as pre_resnet
from keras.applications.mobilenet import preprocess_input as pre_mobile

from ultralytics import YOLO


In [2]:
# Check for GPU:

import tensorflow as tf 
x = tf.config.list_physical_devices('GPU') 
print(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
# Import CNN models and COCO names:

#detector_cv2 = cv2.CascadeClassifier('./others/haarcascade_frontalface_default.xml')     # was just used for testing purposes, didn't yield good results here.

mask_classifier_resnet = load_model('./model/resnet.h5')
mask_classifier_mobile = load_model('./model/mobilenet.h5')
model = YOLO("../project/yolov8/yolov8n.pt")

classesFile = "./others/coco.names"
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [ ]:
# The main program:

detector = face_detection.build_detector("DSFDDetector", confidence_threshold=.7, nms_iou_threshold=.5) 
cap = cv2.VideoCapture("./media/testvid.mp4")

# Initialize Output Video Stream

video_fps = int(cap.get(cv2.CAP_PROP_FPS))
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out_stream = cv2.VideoWriter(
    "./media/video_processed.mp4", 
    cv2.VideoWriter_fourcc('X','V','I','D'),
    video_fps,
    (video_width,video_height))

input_width = 640
input_height = 640




while cap.isOpened():       

    ret,img = cap.read()
    if img is None:
        break

    img2 = img.copy()

    cv2.putText(img2,  "CV Project by Slavka", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Some Lists to work with, which will be reseted at every frame:    
    violate=[]
    violate1=[]
    centroids = []
    coordinates_people = [[(),(),]]
    people_too_close = []
    bounding_boxes = []
    # Using YOLOv8 for predicting people:
    results = model.predict(img)
    result = results[0]
    not_violate = []

    # Append coordinates of detected people only:
    for box in result.boxes:
        class_id = box.cls[0].item()
        conf = round(box.conf[0].item(), 2)
        x0, y0, x1, y1 = [round(i) for i in box.xyxy[0].tolist()]
        
        if conf >= 0.4 and class_id == 0:
            bounding_boxes.append([x0, y0, x1, y1])
            #height = int((y1 - y0))
            width = int((x1-x0))
            safe_distance = int((3.055 * width * 2)/ 1.59)
            coordinates_people.append([ (x0, y0) ,   (x1, y1) ,   safe_distance   ])

            center_x = int((x0+x1) / 2)
            center_y = int((y0+y1) / 2)
            centroids.append((center_x, center_y))

    coordinates_people.pop(0)

    # If minimum 2 persons are detected in the frame: check if distance is okay, if not, add the persons to list people_too_close:
    if len(coordinates_people) >= 2:
        real_distance = dist.cdist(centroids, centroids, metric="euclidean")

        for i in range(len(coordinates_people)):
            for j in range (i+1,len(coordinates_people)):
                if real_distance[i,j] < coordinates_people[i][2] or real_distance[i,j] < coordinates_people[j][2]:
                    #                                                                                                                 (x0, y0)              (x1, y1)        
                    if [coordinates_people[i][0],coordinates_people[i][1]] not in people_too_close: people_too_close.append([coordinates_people[i][0], coordinates_people[i][1]])   
                    if [coordinates_people[j][0],coordinates_people[j][1]] not in people_too_close: people_too_close.append([coordinates_people[j][0], coordinates_people[j][1]])


        # Detect faces only in recognized people, which are too close to each other:
        for idx in range(len(people_too_close)):
            person_rgb = img[people_too_close[idx][0][1]:people_too_close[idx][1][1], people_too_close[idx][0][0]:people_too_close[idx][1][0]]
            
            # Draw Rectangle for testing purpose:
            cv2.rectangle(img2, people_too_close[idx][0], people_too_close[idx][1], (0,255,255), 1)
            person_rgb = cv2.GaussianBlur(person_rgb, (5,5), cv2.BORDER_DEFAULT) 

            detections = detector.detect(person_rgb)
            test_area = 0
            biggest_face = ((0,0),(0,0))
            faces=[]
            face_test = []

            for i in range(len(detections)):
                detection = np.array(detections[i])                     
                detection = np.where(detection<0,0,detection)        
                face_x0 = people_too_close[idx][0][0] + int(detection[0])    
                face_x1 = people_too_close[idx][0][0] + int(detection[2])
                face_y0 = people_too_close[idx][0][1] + int(detection[1])
                face_y1 = people_too_close[idx][0][1] + int(detection[3])

                face_area = int((face_x0 - face_x1) * (face_y0 - face_y1))
                faces.append([ (face_x0, face_y0), (face_x1, face_y1), face_area])

                # Draw Rectangle for testing purpose:
                cv2.rectangle(img, faces[i][0], faces[i][1], (55,0,255), 1)
                
            for j in range(len(faces)):
                if faces[j][2] > test_area:
                    #              x0,y0        x1,y1          area
                    biggest_face=(faces[j][0], faces[j][1])
                    test_area = faces[j][2]


            face_test.append(biggest_face)

            for var in range(len(face_test)):
                try:

                    # # Draw Rectangle for testing purpose:
                    cv2.rectangle(img2, (face_test[var][0]), (face_test[var][1]), (255,255,0), 1)


        # ############################################ RESNET #############################################
                    face_rgb = img[face_test[var][0][1]:face_test[var][1][1], face_test[var][0][0]:face_test[var][1][0],::-1]
                    face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)   
                    face_arr = pre_resnet(face_arr)      
                    face_arr = cv2.GaussianBlur(face_arr, (9,9), cv2.BORDER_DEFAULT)    
                    face_arr = np.expand_dims(face_arr, axis=0)           
                    
                    check_if_mask = mask_classifier_resnet.predict(face_arr)
        # ############################################# RESNET #############################################

        ############################################ MobileNet #############################################

                    # face_rgb = img[face_test[var][0][1]:face_test[var][1][1], face_test[var][0][0]:face_test[var][1][0],::-1]
                    # face_arr = cv2.resize(face_rgb, (224, 224), interpolation=cv2.INTER_NEAREST)    
                    # face_arr = pre_mobile(face_arr)   
                    # face_arr = cv2.GaussianBlur(face_arr, (5,5), cv2.BORDER_DEFAULT)
                    # face_arr = np.expand_dims(face_arr, axis=0)           
                        
                    # check_if_mask = mask_classifier_mobile.predict(face_arr)

        ############################################ MobileNet #############################################

                    if check_if_mask[0][0] > 0.89:
                        violate.append(people_too_close[idx])
                        # Draw Circle for testing purpose:
                        cv2.circle(img2, face_test[var][0], 3, (0, 255,0),2)

                except:
                    continue

    # Draw red bounding box if violating the rules:
    for m in range(len(violate)):
        cv2.rectangle(img2, violate[m][0], violate[m][1], (0,0,255), 2)     

    if ret:
        out_stream.write(img2)

    cv2.imshow("Video", img2)

    if cv2.waitKey(1)==ord('q'):
        break

cap.release()    
out_stream.release()

cv2.destroyAllWindows()